##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Distribution Strategy with Training Loops

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/alpha/tutorials/distribute/training_loops"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/distribute/training_loops.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/r2/tutorials/distribute/training_loops.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This tutorial demonstrates how to use [distribution strategy](https://www.tensorflow.org/guide/distribute_strategy) with custom training loops. We will train a simple CNN model on the fashion MNIST dataset. The fashion MNIST dataset contains 60000 train images of size 28 x 28 and 10000 test images of size 28 x 28.

We are using custom training loops to train our model because they give us flexibility and a greater control on training. Moreover, it is easier to debug the model and the training loop.

In [0]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow
!pip install tf-nightly-2.0-preview
import tensorflow as tf

# Helper libraries
import numpy as np
import os

print(tf.__version__)

## Download the fashion mnist dataset

In [0]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Adding a dimension to the array -> new shape == (28, 28, 1)
# We are doing this because the first layer in our model is a convolutional 
# layer and it requires a 4D input (batch_size, height, width, channels).
# batch_size dimension will be added later on.
train_images = train_images[..., None]
test_images = test_images[..., None]

# Getting the images in [0, 1] range.
train_images = train_images / np.float32(255)
test_images = test_images / np.float32(255)

In [0]:
BUFFER_SIZE = len(train_images)
BATCH_SIZE = 64
EPOCHS = 10
train_steps_per_epoch = len(train_images) // BATCH_SIZE
test_steps_per_epoch = len(test_images) // BATCH_SIZE

## Model Creation

Create a model using `tf.keras.Sequential`. You can also use the Model Subclassing API to do this.

In [0]:
def create_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])

  return model

In [0]:
# Create a checkpoint directory to store the checkpoints.
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

## Create a strategy to distribute the variables and the graph

How does `tf.distribute.MirroredStrategy` strategy work?

*   All the variables and the model graph is replicated on the replicas.
*   Input is evenly distributed across the replicas.
*   Each replica calculates the loss and gradients for the input it received.
*   The gradients are synced across all the replicas by summing them. 
*   After the sync, the same update is made to the copies of the variables on each replica.

Note: You can put all the code below inside a single scope. We are dividing it into several code cells for illustration purposes.


In [0]:
# If the list of devices is not specified in the 
# `tf.distribute.MirroredStrategy` constructor, it will be auto-detected.
strategy = tf.distribute.MirroredStrategy()

In [0]:
print ('Number of devices: {}'.format(strategy.num_replicas_in_sync))

##  Create an input pipeline

`strategy.experimental_make_numpy_iterator` creates an iterator that evenly distributes the data across all the replicas.

You can also achieve this by using tf.data.Dataset. 

```python
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
train_iterator = strategy.make_dataset_iterator(train_dataset)
```

In [0]:
with strategy.scope():
  train_iterator = strategy.experimental_make_numpy_iterator(
      (train_images, train_labels), BATCH_SIZE, shuffle=BUFFER_SIZE)

  test_iterator = strategy.experimental_make_numpy_iterator(
      (test_images, test_labels), BATCH_SIZE, shuffle=None)

## Define the loss function

Normally, on a single machine with 1 GPU/CPU, loss is divided by the number of examples in the batch of input.

*So, how is the loss calculated in distribution strategies?*

> For an example, let's say you have 4 GPU's and a batch size of 64. One batch of input is distributed 
across the replicas (4 GPU's), each replica getting an input of size 16. 

> The model on each replica does a forward pass with its respective input and calculates the loss. Now, instead of dividing the loss by the number of examples in its respective input (16), the loss is divided by the global input size (64).

*Why is this done?*

> This is done because after the gradients are calculated on each replica, they are synced across the replicas by **summing** them.

*How to handle this in TensorFlow?*

> `tf.keras.losses` handles this automatically. 

> If you distribute a custom loss function, don't implement it using `tf.reduce_mean` (which divides by the local batch size), divide the sum by the global batch size:

```python
scale_loss = tf.reduce_sum(loss) * (1. / global_batch_size)
```


In [0]:
with strategy.scope():
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

## Define the metrics to track loss and accuracy

These metrics track the loss and the accuracy. You can use .result() to get the accumulated statistics at any time.

In [0]:
with strategy.scope():
  train_loss = tf.keras.metrics.Mean(name='train_loss')
  test_loss = tf.keras.metrics.Mean(name='test_loss')

  train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='train_accuracy')
  test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='test_accuracy')

## Training loop

In [0]:
with strategy.scope():
  model = create_model()
  
  # Make sure the optimizer is inside the scope so that
  # its variables are distributed
  optimizer = tf.keras.optimizers.Adam()
  
  checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)

  # TODO(yashkatariya): Add tf.function when b/123315763 is resolved
  # Train step to calculate the loss, gradients and backpropagate.
  def train_step(inputs):
    images, labels = inputs

    with tf.GradientTape() as tape:
      predictions = model(images, training=True)
      loss = loss_object(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

  # Test step
  def test_step(inputs):
    images, labels = inputs

    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

  for epoch in range(EPOCHS):
    # TRAIN LOOP

    # TODO(yashkatariya): Remove from for loop after b/123315763 is resolved

    # `experimental_run` replicates the model's and optimizer's variables 
    # on all the replicas and runs the model with the input 
    distributed_train = lambda it: strategy.experimental_run(train_step, it)
    distributed_train = tf.function(distributed_train)

    # Initialize the iterator
    train_iterator.initialize()
    for _ in range(train_steps_per_epoch):
      distributed_train(train_iterator)

    # TEST LOOP
    distributed_test = lambda it: strategy.experimental_run(test_step, it)
    distributed_test = tf.function(distributed_test)

    test_iterator.initialize()
    for _ in range(test_steps_per_epoch):
      distributed_test(test_iterator)
    
    if epoch % 2 == 0:
      checkpoint.save(checkpoint_prefix)

    template = ("Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {},"
                "Test Accuracy: {}")
    print (template.format(epoch+1, train_loss.result(), 
                           train_accuracy.result()*100, test_loss.result(), 
                           test_accuracy.result()*100))
    
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

## Restore the latest checkpoint and test

A model checkpointed with distribution strategy can be restored with and without distribution strategy. 

In [0]:
eval_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='eval_accuracy')

new_model = create_model()
new_optimizer = tf.keras.optimizers.Adam()

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(BATCH_SIZE)

In [0]:
@tf.function
def eval_step(images, labels):
  predictions = new_model(images, training=False)
  eval_accuracy(labels, predictions)

In [0]:
checkpoint = tf.train.Checkpoint(optimizer=new_optimizer, model=new_model)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

for images, labels in test_dataset:
  eval_step(images, labels)

print ('Accuracy after restoring the saved model without strategy: {}'.format(
    eval_accuracy.result()*100))

## Next Steps

Try out the new Distribution Strategy API on your models.